# Backpropagation


### Basic neuron formulas
$$ E = { 1 \over 2 } \sum_{i=1} (y_i - a_i)^2 $$
$$ a_i^L = \sigma(z_i^L) $$
$$ z_i^L = \sum_{j=1}^K ( w_{i,j} * a_j^{L-1} + b_i^L ) $$

where:
- $ L $ - current layer index
- $ (L-1) $ - previous layer index
- $ E $ - error function
- $ a_i $ - activation function
- $ z_i $ - linear function

### Calculating weight and bias derivatives for the last layer

Using the chain rule, we can calculate the derivatives of the weights and biases for the last layer:
$$ { \partial E \over \partial w_{i,j}^L } = { \partial E \over \partial a_i } { \partial a_i \over \partial z_i } { \partial z_i \over \partial w_{i,j} } $$
$$ { \partial E \over \partial b_i^L } = { \partial E \over \partial a_i } { \partial a_i \over \partial z_i } { \partial z_i \over \partial b_i } $$

where:
- $ { \partial E \over \partial a_i^L } = y_i - a_i^L $
- $ { \partial a_i^L \over \partial z_i^L } $ - depends on the $ \sigma $
- $ { \partial z_i^L \over \partial w_{i,j}^L } = a_j^{L-1} $
- $ { \partial z_i^L \over \partial b_i^L } = 1 $

Example individual weights (assuming 2 output, 3 hidden nodes)
$$ \begin{bmatrix}
{ \partial E \over \partial w_{0,0}^L } &
{ \partial E \over \partial w_{0,1}^L } &
{ \partial E \over \partial w_{0,2}^L } \\
{ \partial E \over \partial w_{1,0}^L } &
{ \partial E \over \partial w_{1,1}^L } &
{ \partial E \over \partial w_{1,2}^L }
\end{bmatrix} =
\begin{bmatrix}
(y_0 - a_0^L) ({ \partial a_0^L \over \partial z_0^L }) a_0^{L-1} &
(y_0 - a_0^L) ({ \partial a_0^L \over \partial z_0^L }) a_1^{L-1} &
(y_0 - a_0^L) ({ \partial a_0^L \over \partial z_0^L }) a_2^{L-1} \\
(y_1 - a_1^L) ({ \partial a_1^L \over \partial z_1^L }) a_0^{L-1} &
(y_1 - a_1^L) ({ \partial a_1^L \over \partial z_1^L }) a_1^{L-1} &
(y_1 - a_1^L) ({ \partial a_1^L \over \partial z_1^L }) a_2^{L-1} 
\end{bmatrix} $$

Concise implementation for bias and weight gradient using outer product:
$$ 
W^L =
\begin{bmatrix}
(y_0 - a_0^L) { \partial a_0^L \over \partial z_0^L } \\
(y_1 - a_1^L) { \partial a_1^L \over \partial z_1^L }
\end{bmatrix} 
\begin{bmatrix}
a_0^{L-1} \\
a_1^{L-1} \\
a_2^{L-1}
\end{bmatrix}^T $$

$$ 
b^L =
\begin{bmatrix}
(y_0 - a_0^L) { \partial a_0^L \over \partial z_0^L } \\
(y_1 - a_1^L) { \partial a_1^L \over \partial z_1^L }
\end{bmatrix} 
$$

In [ ]:
# Code example for manually calculating weight gradients for the last layer
import torch

# output layer
output = torch.relu(torch.randn(2)) # 2 output neurons
label = torch.randn(2) # grand truth labels
sigma_derivative = (output > 0).to(torch.float32) # ReLU derivative

# hidden layer
hidden_activation = torch.relu(torch.randn(3)) # 3 hidden neurons

# weight and bias gradients
bias_grad = (output - label) * sigma_derivative
weight_grad = torch.outer(bias_grad, hidden_activation)
print(bias_grad)
print(weight_grad)

tensor([1.7041, 3.5989])
tensor([[0.3509, 0.0000, 1.3461],
        [0.7411, 0.0000, 2.8430]])


### Calculating weight and bias derivatives for the last second to last layer

$$ { \partial E \over \partial w_{i,j}^{L-1} } = \sum_k{ ({ \partial E \over \partial a_k^L } { \partial a_k^L \over \partial z_k^L } { \partial z_k^L \over \partial a_i^{L-1} }) } { \partial a_i^{L-1} \over \partial z_i^{L-1} } { \partial z_i^{L-1} \over \partial w_{i,j}^{L-1} } $$
$$ { \partial E \over \partial b_i^{L-1} } = \sum_k{ ({ \partial E \over \partial a_k^L } { \partial a_k^L \over \partial z_k^L } { \partial z_k^L \over \partial a_i^{L-1} }) } { \partial a_i^{L-1} \over \partial z_i^{L-1} } { \partial z_i^{L-1} \over \partial b_i^{L-1} } $$

where:
- $ { \partial z_k \over \partial a_i^{L-1} } = w_{k,i}^L $
- $ { \partial a_i^{L-1} \over \partial z_i^{L-1} } $ - depends on the $ \sigma $
- $ { \partial z_i^{L-1} \over \partial w_{i,j}^{L-1} } = a_j^{L-2} $
- $ { \partial z_i^{L-1} \over \partial b_i^{L-1} } = 1 $

Example individual weights (assuming 3 hidden nodes in $L-1$, 2 hidden nodes in $ L - 2 $) and $k$ is the number of output nodes:
$$ \begin{bmatrix}
{ \partial E \over \partial w_{0,0}^{L-1} } &
{ \partial E \over \partial w_{0,1}^{L-1} } \\
{ \partial E \over \partial w_{1,0}^{L-1} } &
{ \partial E \over \partial w_{1,1}^{L-1} } \\
{ \partial E \over \partial w_{2,0}^{L-1} } &
{ \partial E \over \partial w_{2,1}^{L-1} }
\end{bmatrix} =
\begin{bmatrix}
(\sum_k{ ( y_k - a_k^L ) {\partial a_k^N \over \partial z_k^N} w_{k,0}^N }) {\partial a_0^{L-1} \over \partial z_0^{L-1} } a_0^{L-2}  &
(\sum_k{ ( y_k - a_k^L ) {\partial a_k^N \over \partial z_k^N} w_{k,0}^N }) {\partial a_0^{L-1} \over \partial z_0^{L-1} } a_1^{L-2} \\
(\sum_k{ ( y_k - a_k^L ) {\partial a_k^N \over \partial z_k^N} w_{k,1}^N }) {\partial a_1^{L-1} \over \partial z_1^{L-1} } a_0^{L-2} &
(\sum_k{ ( y_k - a_k^L ) {\partial a_k^N \over \partial z_k^N} w_{k,1}^N }) {\partial a_1^{L-1} \over \partial z_1^{L-1} } a_1^{L-2} \\
(\sum_k{ ( y_k - a_k^L ) {\partial a_k^N \over \partial z_k^N} w_{k,2}^N }) {\partial a_2^{L-1} \over \partial z_2^{L-1} } a_0^{L-2} &
(\sum_k{ ( y_k - a_k^L ) {\partial a_k^N \over \partial z_k^N} w_{k,2}^N }) {\partial a_2^{L-1} \over \partial z_2^{L-1} } a_1^{L-2} \\
\end{bmatrix} $$

Concise implementation for bias and weight gradient using outer product:
$$ 
W^{L-1} =
(
(W^L)^T
\begin{bmatrix}
(y_0 - a_0^L) { \partial a_0^L \over \partial z_0^L } \\
(y_1 - a_1^L) { \partial a_1^L \over \partial z_1^L }
\end{bmatrix}
)
{\partial a_0^{L-1} \over \partial z_0^{L-1} }
\otimes
\begin{bmatrix}
a_0^{L-2} \\
a_1^{L-2}
\end{bmatrix}^T $$

$$ 
b^{L-1} =
(
(W^L)^T
\begin{bmatrix}
(y_0 - a_0^L) { \partial a_0^L \over \partial z_0^L } \\
(y_1 - a_1^L) { \partial a_1^L \over \partial z_1^L }
\end{bmatrix}
)
{\partial a_0^{L-1} \over \partial z_0^{L-1} }
$$

In [43]:
# Code example for manually calculating weight gradients for the second last layer
import torch

# output layer
output = torch.relu(torch.randn(2)) # 2 output neurons
label = torch.randn(2) # grand truth labels
output_sigma_derivative = (output > 0).to(torch.float32) # ReLU derivative

# hidden layer
hidden_activation = torch.relu(torch.randn(3)) # 3 hidden neurons
hidden_sigma_derivative = (hidden_activation > 0).to(torch.float32) # ReLU derivative
hidden_weight = torch.randn(2, 3) # weights from hidden to output layer

# previous hidden layer
prev_hidden_activation = torch.relu(torch.randn(2)) # 3 hidden neurons

# weight and bias gradients
bias_grad = (hidden_weight.T @ ((output - label) * output_sigma_derivative)) * hidden_sigma_derivative
weight_grad = torch.outer(bias_grad, prev_hidden_activation)
print(bias_grad)
print(weight_grad)

tensor([ 0.8434, -0.1364, -0.0000])
tensor([[ 0.6257,  0.4763],
        [-0.1012, -0.0770],
        [-0.0000, -0.0000]])


### Generic formula for Nth layer

$$ 
b^{N} =
(
(W^{N+1})^T
{\partial E \over \partial b^{N+1}}
)
{\partial a_0^{N} \over \partial z_0^{N} }
$$

$$
W^{N} =
(
b^{N}
)
\otimes
(a^{N-1})^T
$$

NOTE: We can reuse calculations for the current layer's bias during weight's calculation.

In [ ]:
# Example of backpropagation step manually vs using autograd
# NN model: 
#   * input (2 neurons)
#   * 3x hidden layer (3,2,3 neurons) - ReLU activation
#   * output (2 neurons)
import torch

# Define model parameters and input
torch.manual_seed(1337)
input = torch.tensor([0.5, -0.2])

weights = [
    torch.randn(3, 2, requires_grad=True), # input to hidden1
    torch.randn(2, 3, requires_grad=True), # hidden1 to hidden2
    torch.randn(3, 2, requires_grad=True), # hidden2 to hidden3
    torch.randn(2, 3, requires_grad=True), # hidden3 to output
]

biases = [
    torch.randn(3, requires_grad=True), # hidden1
    torch.randn(2, requires_grad=True), # hidden2
    torch.randn(3, requires_grad=True), # hidden3
    torch.randn(2, requires_grad=True), # output
]

label = torch.tensor([1.0, 0.0])

# Forward pass
activations = [input]
for i, (W, b) in enumerate(zip(weights, biases)):
    is_last = (i == len(weights) - 1)
    z = W @ activations[-1] + b
    activations.append(
        z if is_last else torch.relu(z) # skip ReLU for the output layer
    )

output = activations.pop()

# Backpropagation using manual calculations
manual_weight_grads = list()
manual_bias_grads = list()
with torch.no_grad():
    loss_grad = 2/len(label) * (output - label) # MSE loss gradient
    bias_grad = loss_grad.clone()
    for W in reversed(weights):
        a = activations.pop()
        weight_grad = torch.outer(bias_grad, a)
        manual_weight_grads.append(weight_grad.clone())
        manual_bias_grads.append(bias_grad.clone())

        # for the next iteration
        sigma_derivative = (a > 0).to(torch.float32) # ReLU derivative
        bias_grad = (W.T @ bias_grad) * sigma_derivative

    manual_weight_grads.reverse()
    manual_bias_grads.reverse()

# Backpropagation using autograd
loss = torch.nn.functional.mse_loss(output, label)
loss.backward()

# Compare results
for i, (W, b) in enumerate(zip(weights, biases)):
    print(f"Layer {i+1} weight grad difference: {torch.linalg.vector_norm(W.grad - manual_weight_grads[i])}")
    print(f"Layer {i+1} bias grad difference: {torch.linalg.vector_norm(b.grad - manual_bias_grads[i])}")

Layer 1 weight grad difference: 0.0
Layer 1 bias grad difference: 0.0
Layer 2 weight grad difference: 0.0
Layer 2 bias grad difference: 0.0
Layer 3 weight grad difference: 0.0
Layer 3 bias grad difference: 0.0
Layer 4 weight grad difference: 0.0
Layer 4 bias grad difference: 0.0
